In [17]:
import functools
import operator
import itertools
import math
import sys
import json
import re
from typing import Tuple

data = open("input.txt").read().splitlines()[0]

moves = [(-1,0) if m == "<" else (1,0) for m in data]

def add(a,b):
    return (a[0] + b[0], a[1] + b[1])

class Block():
    def __init__(self, coords):
        self.coords = coords
    def move(self, offset, board):
        next = [add(offset, c) for c in self.coords]
        if any([board.collides(c) for c in next]):
            return False
        self.coords = next
        return True
    def rows(self):
        rows = set()
        for _, y in self.coords:
            rows.add(y)
        return rows

class Board():
    def __init__(self, width, moves, blocks):
        self.width = width
        self.min_x = 0
        self.max_x = width - 1
        self.floor = 0
        self.occupied = set([(x,self.floor) for x in range(0, width)])
        self.rows = []
        self.row_strings = ['-' * self.width]
        self.row_map = {}
        self.row_last_changed_map = {}
        self.block_count = 0
        self.min_repeat_height = (len(moves)*len(blocks))/math.gcd(len(moves),len(blocks))
        self.repeat_data = None
    def collides(self, coords):
        x, y = coords
        return x < self.min_x or \
               x > self.max_x or \
               coords in self.occupied        
    def lock_block(self, block):
        self.block_count += 1
        for c in block.coords:
            self.lock_coords(c)
        top_row = self.row_strings[self.floor]
        row_indices = self.row_map[top_row]
        if self.repeat_data or self.floor < self.min_repeat_height:
            return
        max_repeat_start = self.floor - math.floor(self.floor / 3)
        for candidate in row_indices:
            if candidate > max_repeat_start:
                continue
            if self.find_repeats(self.floor, candidate):
                print(f"Found repeats of length {self.floor - candidate} for rows {self.floor} and {candidate}")
                print(f"Last changes: ")
                floor_last_change = self.row_last_changed_map[self.floor]
                candidate_last_change = self.row_last_changed_map[candidate]
                print(f"  {self.floor} => {floor_last_change}")
                print(f"  {candidate} => {candidate_last_change}")
                self.repeat_data = (
                    floor_last_change,
                    candidate_last_change,
                    self.floor,
                    candidate
                )

    def find_repeats(self, a, b):
        for offset in range(0, a - b + 1):
            a2 = self.row_strings[a - offset]
            b2 = self.row_strings[b - offset]
            if a2 != b2:
                return False
        return True

    def lock_coords(self, coords):
        x,y = coords
        self.occupied.add(coords)
        while len(self.rows) <= y:
            self.rows.append(["." for _ in range(self.width)])
            self.row_strings.append('.' * self.width)
        row = self.rows[y]
        row[x] = "#"
        row_string = "".join(row)
        self.row_strings[y] = row_string
        prev_row_string = self.row_strings[y]
        prev_row_indices = self.row_map.get(prev_row_string, [])
        row_indices = self.row_map.get(row_string, [])
        row_indices.append(y)
        self.row_map[row_string] = row_indices
        self.floor = max(coords[1], self.floor)
        self.row_last_changed_map[y] = self.block_count

    def dump(self, message = "") -> str:
        print("---------------------------------------")
        print(message)
        for x in range(len(self.row_strings) - 1, -1, -1):
            print(f"{x:10}|{self.row_strings[x]}|")   
        print(message)
    

def factory(coords):
    return lambda floor: Block([add((2, floor + 4), c) for c in coords])

blocks = [
    factory([(0,0),(1,0),(2,0),(3,0)]),
    factory([(1,2),
      (0,1), (1,1), (2,1),
             (1,0)]),
    factory([       (2,2),
                    (2,1),
      (0,0), (1,0), (2,0)]),
    factory([(0,3),
             (0,2),
             (0,1),
             (0,0)]),
    factory([(0,1),(1,1),
             (0,0),(1,0)])
]

block_seq = itertools.cycle(blocks)
move_seq = itertools.cycle(moves)

board = Board(7, moves, blocks)

target = 1000000000000
c = 0
elided_rows = 0
while c < target:
    fn = next(block_seq)
    block = fn(board.floor)
    block.move(next(move_seq), board)
    while block.move((0,-1), board):
        block.move(next(move_seq), board)
    board.lock_block(block)

    if board.repeat_data and elided_rows == 0:
        end_block, start_block, end_row, start_row = board.repeat_data
        repeat_block_count = end_block - start_block
        repeat_height = end_row - start_row
        print(f"Eliding blocks given repeat block count {repeat_block_count} and repeat height {repeat_height}")
        elided_cycles = math.floor((target - c) / repeat_block_count)
        c += repeat_block_count * elided_cycles
        elided_rows = repeat_height * elided_cycles

    c += 1

print(f"actual floor: {board.floor}")
print(f"floor + elided rows: {board.floor + elided_rows}")








move count = 10091
---------------------------------------

         0|-------|

Found repeats of length 23823 for rows 50465 and 26642
Last changes: 
  50465 => 32228
  26642 => 17018
Found repeats of length 21176 for rows 50465 and 29289
Last changes: 
  50465 => 32228
  29289 => 18708
Found repeats of length 18529 for rows 50465 and 31936
Last changes: 
  50465 => 32228
  31936 => 20398
Eliding blocks given repeat block count 11830 and repeat height 18529
actual floor: 51173
floor + elided rows: 1566272189352
